## LISTADO PARTES ##

In [3]:
import numpy as np
import pandas as pd
import re
import unicodedata

pd.options.mode.copy_on_write = True # CoW por defecto a partir de Pandas 3.0.0

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


In [4]:
# carga de data set
df_raw = pd.read_csv(
    "./datasets/ListadoPartes.csv",
    low_memory=False
)
# Se usa rw porque es necesario leer el dataset y aplicar cambios durante su limpieza.

In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NIF                    631 non-null    object 
 1   Nombre                 631 non-null    object 
 2   Apellido1              631 non-null    object 
 3   Apellido2              630 non-null    object 
 4   FechaParteAccidente    631 non-null    object 
 5   FechaAltaParte         631 non-null    object 
 6   Codigo                 631 non-null    int64  
 7   DescripcionParte       631 non-null    object 
 8   Actividad              631 non-null    object 
 9   Lugar                  631 non-null    object 
 10  FechaRecepcion         631 non-null    object 
 11  CentroUrgencias        257 non-null    object 
 12  Helicoptero            0 non-null      float64
 13  MacizoMontañoso        628 non-null    object 
 14  NombreParaje           628 non-null    object 
 15  Entren

In [6]:
df_raw.shape, df_raw.columns

((631, 26),
 Index(['NIF', 'Nombre', 'Apellido1', 'Apellido2', 'FechaParteAccidente', 'FechaAltaParte', 'Codigo', 'DescripcionParte', 'Actividad', 'Lugar', 'FechaRecepcion', 'CentroUrgencias', 'Helicoptero', 'MacizoMontañoso', 'NombreParaje', 'Entrenamiento', 'ActividadPersonal', 'ActividadOrganizada', 'Festivo', 'NResponsables', 'TamañoGrupo', 'CentroHospitalizacion', 'DescripcionGrado', 'Pais', 'Provincia', 'TipoAccidente'], dtype='object'))

In [7]:
df = df_raw.copy()

In [8]:
df.head(10)

,NIF,Nombre,Apellido1,Apellido2,FechaParteAccidente,FechaAltaParte,Codigo,DescripcionParte,Actividad,Lugar,FechaRecepcion,CentroUrgencias,Helicoptero,MacizoMontañoso,NombreParaje,Entrenamiento,ActividadPersonal,ActividadOrganizada,Festivo,NResponsables,TamañoGrupo,CentroHospitalizacion,DescripcionGrado,Pais,Provincia,TipoAccidente
0,05277296S,MARIA,NAZAR,ENRIQUEZ DE SALAMANCA,1/1/24 0:00,1/2/24 12:04,745025061,Caida esquiando,Esquí Alpino,Baqueira Beret,1/2/24 0:00,NaN,NaN,Estacion esqui Baqueira,Baqueira Beret,False,True,False,False,0,0,NaN,Leve,España,LLEIDA,Tropiezos (caída al mismo nivel)
1,50775542Y,PABLO,SAN JOSE,PEÑA,1/1/24 14:00,1/2/24 10:48,745024991,"Montando en mtb, mi pie salio del pedal y se g...",BTT,El Pardo,1/2/24 10:48,NaN,NaN,El Pardo,El Pardo,True,False,False,False,1,0,NaN,NaN,España,MADRID,Otro: Golpe en el pie
2,11851622K,PAULA,ARQUERO,CONTRERAS,1/2/24 17:00,1/18/24 9:27,745038371,Caída al ir andando con la bicicleta y tropeza...,BTT,Carretera de la republica - Dehesas de Cercedilla,1/18/24 0:00,NaN,NaN,Sistema Central,Sierra de Guadarrama,False,True,False,False,0,0,NaN,Leve,España,MADRID,Tropiezos (caída al mismo nivel)
3,50208024J,MIGUEL,PEREZ,PANIAGUA,1/4/24 0:00,1/15/24 16:09,745034561,Caída de manos. Golpe en la mano derecha,Carreras por montaña,La pedriza,1/15/24 0:00,NaN,NaN,Central,La Pedriza,True,False,False,False,1,1,NaN,Leve,España,MADRID,Colisión con rocas
4,09141804V,VICTOR,MORALEDA,ALVAREZ,1/4/24 19:30,1/4/24 23:32,745025901,"Accidente en rocódromo,haciendo dry tooling, c...",Rocódromo,Rocódromo indoorwall,1/4/24 23:32,Hospital de Getafe,NaN,No,No,True,False,False,False,1,9,NaN,NaN,España,MADRID,Otro: Corte con el filo del crampón
5,04613683K,FRANCISCO JAVIER,DIAZ,OLAYA,1/7/24 7:30,1/8/24 10:34,745026201,Practicando carrera por el campo a los 30 minu...,Carreras por montaña,Indefinido,1/8/24 0:00,Clinica Cemtro,NaN,El pardo,Camino de santiago de madrid,False,True,False,False,1,1,NaN,Leve,España,MADRID,Problemas físicos
6,06010495C,DANIEL,ELORZ,RUIZ,1/7/24 12:00,1/7/24 17:00,60676,Molestias muy fuertes en la cadera obligando a...,Carreras por montaña,El Escorial,1/7/24 17:00,Ninguno,NaN,Monte Abantos,Monte Abantos,True,False,False,False,2,10,NaN,NaN,España,MADRID,Problemas físicos
7,05338813F,LUCIA,GUICHOT,MARTIN,1/8/24 0:00,1/10/24 12:21,745027371,Avalancha,Esquí de montaña/Splitboard,Vall de Ruda,1/10/24 0:00,Viella,NaN,Pirineo,Aran,False,True,False,False,1,6,Viella,Grave,España,LLEIDA,Avalancha de nieve
8,50312050X,PATRICIA,LOPEZ,YEPES,1/8/24 0:00,1/15/24 14:36,745026961,Dolor intenso en tobillo derecho y rodilla izda.,Snowboard,Dossau,1/15/24 0:00,Clínica Vidal y hospital Viehla,NaN,Pirineo Catalan,Baqueira,False,True,False,False,1,1,NaN,Leve,España,MADRID,Tropiezos (caída al mismo nivel)
9,02087155C,JOSEFA,CARCHANO,CUESTA,1/8/24 11:30,1/16/24 12:49,745028921,Caida esquiando en la estación de Baqueira Ber...,Esquí Alpino,PISTA DE LA CABANHA,1/16/24 0:00,Centro Médico Baqueira - .Clinica VIDAL,NaN,PIRINEOS,ESTACION DE ESQUI BAQUEIRA BERET,False,True,False,False,0,0,NaN,Leve,España,LLEIDA,Tropiezos (caída al mismo nivel)


In [9]:
# Normalización de nombres de columnas
def normalizar_texto(texto):
    texto = texto.lower().strip()
    texto = unicodedata.normalize('NFKD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')
    texto = texto.replace(' ', '_')
    return texto

df.columns = [normalizar_texto(col) for col in df.columns]

In [10]:
#eliminar duplicados exactos
df = df.drop_duplicates()


In [11]:
#Duplicados lógicos (siniestro repetido)
df = df.drop_duplicates(
    subset=[
        'nif',
        'fechaparteaccidente',
        'actividad',
        'tipoaccidente'
    ],
    keep='first'
)


In [12]:
# Eliminación de columnas irrelevantes
cols_eliminar = [
    'nombre',
    'apellido1',
    'apellido2',
    'nombreparaje',
    'centrourgencias',
    'centrohospitalizacion',
    'codigo'
]

df.drop(columns=cols_eliminar, inplace=True, errors='ignore')

df.columns


Index(['nif', 'fechaparteaccidente', 'fechaaltaparte', 'descripcionparte', 'actividad', 'lugar', 'fecharecepcion', 'helicoptero', 'macizomontanoso', 'entrenamiento', 'actividadpersonal', 'actividadorganizada', 'festivo', 'nresponsables', 'tamanogrupo', 'descripciongrado', 'pais', 'provincia', 'tipoaccidente'], dtype='object')

In [14]:
#Conversión de columnas de fecha
cols_fecha = [
    'fechaparteaccidente',
    'fechaaltaparte',
    'fecharecepcion'
]

for col in cols_fecha:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')


In [15]:
#nulos antes 
nulos_antes = df.isna().mean().sort_values(ascending=False) * 100
nulos_antes


helicoptero            100.00
descripciongrado         6.72
macizomontanoso          0.48
fechaaltaparte           0.00
fechaparteaccidente      0.00
nif                      0.00
descripcionparte         0.00
fecharecepcion           0.00
lugar                    0.00
entrenamiento            0.00
actividad                0.00
actividadpersonal        0.00
actividadorganizada      0.00
nresponsables            0.00
festivo                  0.00
tamanogrupo              0.00
pais                     0.00
provincia                0.00
tipoaccidente            0.00
dtype: float64

In [16]:
# Eliminación de columnas con >70% de nulos
cols_muy_nulas = nulos_antes[nulos_antes > 70].index.tolist()

df.drop(columns=cols_muy_nulas, inplace=True)

cols_muy_nulas


['helicoptero']

In [17]:
# Normalización semántica de variables binarias

cols_binarias = [
    'helicoptero',
    'entrenamiento',
    'actividadpersonal',
    'actividadorganizada',
    'festivo'
]

for col in cols_binarias:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.lower()
            .replace({
                'si': 'si',
                'sí': 'si',
                'no': 'no',
                'nan': np.nan
            })
        )
# Se realizó una normalización semántica de las variables binarias para unificar distintas representaciones textuales 
# de un mismo significado y garantizar la validez del análisis estadístico.

In [18]:
#Normalización de gravedad del accidente

df['descripciongrado'] = (
    df['descripciongrado']
    .astype(str)
    .str.lower()
    .str.strip()
)

df['descripciongrado'].value_counts()


descripciongrado
leve      521
grave      56
nan        42
mortal      6
Name: count, dtype: int64

In [19]:
#Limpieza del identificador para el merge

df['nif'] = df['nif'].astype(str).str.strip()  #Esta celda asegura que licencias y partes se unan correctamente


In [20]:
nulos_despues = df.isna().mean().sort_values(ascending=False) * 100

pd.DataFrame({
    'nulos_antes_%': nulos_antes,
    'nulos_despues_%': nulos_despues
})


,nulos_antes_%,nulos_despues_%
actividad,0.00,0.00
actividadorganizada,0.00,0.00
actividadpersonal,0.00,0.00
descripciongrado,6.72,0.00
descripcionparte,0.00,0.00
entrenamiento,0.00,0.00
fechaaltaparte,0.00,0.00
fechaparteaccidente,0.00,0.00
fecharecepcion,0.00,0.00
festivo,0.00,0.00


In [21]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 625 entries, 0 to 630
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   nif                  625 non-null    object        
 1   fechaparteaccidente  625 non-null    datetime64[ns]
 2   fechaaltaparte       625 non-null    datetime64[ns]
 3   descripcionparte     625 non-null    object        
 4   actividad            625 non-null    object        
 5   lugar                625 non-null    object        
 6   fecharecepcion       625 non-null    datetime64[ns]
 7   macizomontanoso      622 non-null    object        
 8   entrenamiento        625 non-null    object        
 9   actividadpersonal    625 non-null    object        
 10  actividadorganizada  625 non-null    object        
 11  festivo              625 non-null    object        
 12  nresponsables        625 non-null    int64         
 13  tamanogrupo          625 non-null    int

,nif,fechaparteaccidente,fechaaltaparte,descripcionparte,actividad,lugar,fecharecepcion,macizomontanoso,entrenamiento,actividadpersonal,actividadorganizada,festivo,nresponsables,tamanogrupo,descripciongrado,pais,provincia,tipoaccidente
0,05277296S,2024-01-01 00:00:00,2024-01-02 12:04:00,Caida esquiando,Esquí Alpino,Baqueira Beret,2024-01-02 00:00:00,Estacion esqui Baqueira,false,true,false,false,0,0,leve,España,LLEIDA,Tropiezos (caída al mismo nivel)
1,50775542Y,2024-01-01 14:00:00,2024-01-02 10:48:00,"Montando en mtb, mi pie salio del pedal y se g...",BTT,El Pardo,2024-01-02 10:48:00,El Pardo,true,false,false,false,1,0,nan,España,MADRID,Otro: Golpe en el pie
2,11851622K,2024-01-02 17:00:00,2024-01-18 09:27:00,Caída al ir andando con la bicicleta y tropeza...,BTT,Carretera de la republica - Dehesas de Cercedilla,2024-01-18 00:00:00,Sistema Central,false,true,false,false,0,0,leve,España,MADRID,Tropiezos (caída al mismo nivel)
3,50208024J,2024-01-04 00:00:00,2024-01-15 16:09:00,Caída de manos. Golpe en la mano derecha,Carreras por montaña,La pedriza,2024-01-15 00:00:00,Central,true,false,false,false,1,1,leve,España,MADRID,Colisión con rocas
4,09141804V,2024-01-04 19:30:00,2024-01-04 23:32:00,"Accidente en rocódromo,haciendo dry tooling, c...",Rocódromo,Rocódromo indoorwall,2024-01-04 23:32:00,No,true,false,false,false,1,9,nan,España,MADRID,Otro: Corte con el filo del crampón


In [22]:
df.shape

(625, 18)

In [23]:
df.to_csv("./datasets/partes_limpio.csv", index=False)
